# Download Dataset

In [9]:
# !pip install gdown

In [ ]:
# import gdown
# import zipfile
# import os

# def download_and_extract_from_gdrive(gdrive_url, extract_to="extracted"):
#     # Convert shared drive URL to direct download URL
#     file_id = gdrive_url.split("/d/")[1].split("/")[0]
#     download_url = f"https://drive.google.com/uc?id={file_id}"

#     # Create output folder
#     os.makedirs(extract_to, exist_ok=True)

#     zip_path = os.path.join(extract_to, "downloaded.zip")

#     print("[INFO] Downloading ZIP from Google Drive...")
#     gdown.download(download_url, zip_path, quiet=False)

#     print("[INFO] Extracting ZIP...")
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)

#     # Optionally, remove the ZIP file after extraction
#     os.remove(zip_path)

#     print(f"[DONE] Extracted files to: {extract_to}")

# # --------- Usage ----------
# gdrive_url = "https://drive.google.com/file/d/12cbrTaBAMIsuU-mwAA7IgDk9wSLC9cC-/view?usp=sharing"
# download_and_extract_from_gdrive(gdrive_url, extract_to="dataset")

[INFO] Downloading ZIP from Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=12cbrTaBAMIsuU-mwAA7IgDk9wSLC9cC-
From (redirected): https://drive.google.com/uc?id=12cbrTaBAMIsuU-mwAA7IgDk9wSLC9cC-&confirm=t&uuid=8c3c49cd-c5fe-4404-8e0f-d4cff3377313
To: c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\downloaded.zip
100%|██████████| 5.21G/5.21G [02:48<00:00, 30.9MB/s]


[INFO] Extracting ZIP...
[DONE] Extracted files to: dataset


# Convert dataset into yolo format

In [4]:
import os
from PIL import Image
from tqdm.auto import tqdm

def convert_annotation(input_path, output_path, image_path, stats):
    try:
        image = Image.open(image_path)
        image_width, image_height = image.size
    except Exception as e:
        print(f"⚠️  Could not open image: {image_path} — {e}")
        stats["missing_image"] += 1
        return

    with open(input_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            parts = line.strip().split(",")
            if len(parts) < 6:
                stats["malformed"] += 1
                continue

            x, y, w, h = map(int, parts[:4])
            class_id_raw = int(parts[5])
            stats["total"] += 1

            if class_id_raw not in [0, 1, 2, 3]:
                stats["skipped"][class_id_raw] += 1
                continue

            stats["converted"] += 1

            x_center = (x + w / 2) / image_width
            y_center = (y + h / 2) / image_height
            w_norm = w / image_width
            h_norm = h / image_height

            outfile.write(f"{class_id_raw} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

In [5]:
from collections import defaultdict
from glob import glob
from tqdm.auto import tqdm


root_dir = "./dataset/UAVDT-2024"
annotation_paths = glob(os.path.join(root_dir, "M*/annotations/*.txt"))
total_files = len(annotation_paths)

stats = {
    "total": 0,
    "converted": 0,
    "malformed": 0,
    "missing_image": 0,
    "skipped": defaultdict(int)
}

print(f"🔄 Converting {total_files} annotation files to YOLO format with dynamic image size...")

for anno_path in tqdm(annotation_paths, desc="Converting", unit="file"):
    sequence_dir = os.path.dirname(os.path.dirname(anno_path))  # Mxxxx
    file_name = os.path.basename(anno_path)

    label_dir = os.path.join(sequence_dir, "labels")
    os.makedirs(label_dir, exist_ok=True)

    label_path = os.path.join(label_dir, file_name)

        # 🔍 Construct image path
    image_name = file_name.replace(".txt", ".jpg")
    image_path = os.path.join(sequence_dir, "images", image_name)

    convert_annotation(anno_path, label_path, image_path, stats)

print("\n✅ Conversion complete.")
print(f"📊 Total boxes:     {stats['total']}")
print(f"✅ Converted boxes: {stats['converted']}")
print(f"❌ Skipped boxes:   {sum(stats['skipped'].values())}")
for cls, count in sorted(stats["skipped"].items()):
    print(f"   - Skipped class {cls}: {count}")
print(f"⚠️ Malformed lines: {stats['malformed']}")
print(f"🖼️  Missing images: {stats['missing_image']}")


🔄 Converting 30946 annotation files to YOLO format with dynamic image size...


Converting:   0%|          | 0/30946 [00:00<?, ?file/s]


✅ Conversion complete.
📊 Total boxes:     868139
✅ Converted boxes: 868139
❌ Skipped boxes:   0
⚠️ Malformed lines: 0
🖼️  Missing images: 0


# Split into train and test sets

In [6]:
# save this as split_uavdt_train_val.py

import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

def copy_split_sequences(src_root, dst_root, train_ratio=0.8):
    all_sequences = sorted(glob(os.path.join(src_root, "M*")))
    train_seqs, val_seqs = train_test_split(all_sequences, train_size=train_ratio, random_state=42)

    for split_name, split_list in zip(['train', 'val'], [train_seqs, val_seqs]):
        for seq_path in tqdm(split_list):
            images_src = os.path.join(seq_path, "images")
            labels_src = os.path.join(seq_path, "labels")
          
            images_dst = os.path.join(dst_root, split_name, "images")
            labels_dst = os.path.join(dst_root, split_name, "labels")

            os.makedirs(images_dst, exist_ok=True)
            os.makedirs(labels_dst, exist_ok=True)

            for img_file in glob(os.path.join(images_src, "*.jpg")):
                shutil.copy(img_file, os.path.join(images_dst, os.path.basename(img_file)))
                
            for label_file in glob(os.path.join(labels_src, "*.txt")):
                shutil.copy(label_file, os.path.join(labels_dst, os.path.basename(label_file)))
            

    print("✅ Split complete into train/ and val/")


In [7]:
src_root = "./dataset/UAVDT-2024"
dst_root = "./dataset/UAVDT-processed"
copy_split_sequences(src_root, dst_root)


  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

✅ Split complete into train/ and val/


# Dataset visulaization

In [10]:
# import os
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from PIL import Image

# # Paths
# image_dir = "./dataset/UAVDT-processed/train/images"
# label_dir = "./dataset/UAVDT-processed/train/labels"

# # Get first 5 image files
# image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])[:20]

# # Class ID to name mapping (modify this as needed)
# class_id_to_name = {
#     0: "car",
#     1: "truck",
#     2: "bus",
#     3: "van"  # if used
# }

# # Visualization loop
# for img_name in image_files:
#     img_path = os.path.join(image_dir, img_name)
#     label_path = os.path.join(label_dir, img_name.replace(".jpg", ".txt"))

#     # Open image
#     image = Image.open(img_path)
#     width, height = image.size

#     # Plot setup
#     fig, ax = plt.subplots(1)
#     ax.imshow(image)

#     # Read and draw bounding boxes
#     if os.path.exists(label_path):
#         with open(label_path, 'r') as f:
#             for line in f.readlines():
#                 class_id, x_center, y_center, w, h = map(float, line.strip().split())

#                 # Convert YOLO format to (x1, y1) top-left corner
#                 x1 = (x_center - w / 2) * width
#                 y1 = (y_center - h / 2) * height
#                 box_w = w * width
#                 box_h = h * height

#                 rect = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor='r', facecolor='none')
#                 ax.add_patch(rect)
#                 label = class_id_to_name.get(int(class_id), f"Class {int(class_id)}")
#                 ax.text(x1, y1 - 5, label, color='white', bbox=dict(facecolor='red', alpha=0.5))

#     ax.set_title(f"Image: {img_name}")
#     plt.axis('off')
#     plt.show()


# Training Yolo Model on Datsets

In [13]:
import os
from ultralytics import YOLO

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

model = YOLO("yolov8n.pt")

model.train(
    data="uavdt.yaml",
    epochs=1,
    imgsz=640,
    batch=32,
    show=False,
    name="yolov8-uavdt",
    project="runs/train"
)


Ultralytics 8.3.107  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=uavdt.yaml, epochs=1, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=yolov8-uavdt3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

train: Scanning D:\2_Code\10_ArtificialIntelligence\1_DeepLearning\1_DeepLearning_HandsOn_Pytorch\7_MTP\Pipeline\dataset\UAVDT-processed\train\labels.cache... 2035 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2035/2035 [00:00<?, ?it/s]
val: Scanning D:\2_Code\10_ArtificialIntelligence\1_DeepLearning\1_DeepLearning_HandsOn_Pytorch\7_MTP\Pipeline\dataset\UAVDT-processed\val\labels.cache... 1101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1101/1101 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8-uavdt3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\train\yolov8-uavdt3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.549      1.958     0.9539        425        640: 100%|██████████| 64/64 [10:07<00:00,  9.49s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [01:28<00:00,  4.92s/it]


                   all       1101      67296     0.0262      0.162     0.0419     0.0191

1 epochs completed in 0.194 hours.
Optimizer stripped from runs\train\yolov8-uavdt3\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\yolov8-uavdt3\weights\best.pt, 6.2MB

Validating runs\train\yolov8-uavdt3\weights\best.pt...
Ultralytics 8.3.107  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:59<00:00,  3.30s/it]


                   all       1101      67296     0.0262      0.162     0.0418      0.019
                   car       1101      53644      0.102      0.225      0.161     0.0741
                 truck        907       3904   0.000513    0.00615   0.000259   4.42e-05
                   bus        169        169   0.000738      0.402    0.00402    0.00133
               vehicle       1079       9579    0.00211     0.0158    0.00133   0.000524
Speed: 1.2ms preprocess, 41.6ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs\train\yolov8-uavdt3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021644AA00B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

# Model Metrics

In [17]:
import os
import json
from ultralytics import YOLO

# Path to root directory of your trained runs
root_dir = "runs/train"

# Get all best.pt weights
best_weights = []

for subdir in os.listdir(root_dir):
    full_path = os.path.join(root_dir, subdir, "weights", "best.pt")
    if os.path.isfile(full_path):
        best_weights.append(full_path)

# Use the first found model (you can loop for multiple if needed)
path = best_weights[0]
print(f"Evaluating model: {path}")

# Load and validate the model
model = YOLO(path)
metrics = model.val()

# === Extracted metrics ===
results_dict = metrics.results_dict
maps = metrics.maps
names = metrics.names
save_dir = metrics.save_dir

# === Print key metrics ===
print("\n📊 Validation Metrics:")
for k, v in results_dict.items():
    print(f"{k:<25}: {v:.4f}")

# === Print per-class mAP50 ===
print("\n🎯 Per-Class mAP@0.5:")
for i, name in names.items():
    print(f"{name:<10}: mAP50 = {maps[i]:.6f}")

# === Save metrics to JSON ===
json_path = os.path.join(save_dir, "metrics_summary.json")
with open(json_path, "w") as f:
    json.dump(results_dict, f, indent=4)
print(f"\n✅ Saved metrics to {json_path}")

# === Save per-class mAP to text ===
txt_path = os.path.join(save_dir, "per_class_map.txt")
with open(txt_path, "w") as f:
    for i, name in names.items():
        f.write(f"{name}: mAP50 = {maps[i]:.6f}\n")
print(f"✅ Saved per-class mAP to {txt_path}")


Evaluating model: runs/train\yolov8-uavdt3\weights\best.pt
Ultralytics 8.3.107  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\2_Code\10_ArtificialIntelligence\1_DeepLearning\1_DeepLearning_HandsOn_Pytorch\7_MTP\Pipeline\dataset\UAVDT-processed\val\labels.cache... 1101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1101/1101 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 69/69 [00:50<00:00,  1.37it/s]


                   all       1101      67296     0.0262      0.162     0.0418      0.019
                   car       1101      53644      0.102      0.225      0.161     0.0741
                 truck        907       3904   0.000513    0.00615   0.000259   4.42e-05
                   bus        169        169   0.000738      0.402    0.00402    0.00133
               vehicle       1079       9579    0.00211     0.0158    0.00133   0.000524
Speed: 0.7ms preprocess, 33.7ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\val2

📊 Validation Metrics:
metrics/precision(B)     : 0.0262
metrics/recall(B)        : 0.1624
metrics/mAP50(B)         : 0.0418
metrics/mAP50-95(B)      : 0.0190
fitness                  : 0.0213

🎯 Per-Class mAP@0.5:
car       : mAP50 = 0.074076
truck     : mAP50 = 0.000044
bus       : mAP50 = 0.001331
vehicle   : mAP50 = 0.000524

✅ Saved metrics to runs\detect\val2\metrics_summary.json
✅ Saved per-class mAP to runs\detect\val2\per_cla

# Save Predictions as a new datsets and get metrics

In [1]:
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm
import shutil
import os

root_dir = "runs/train"

best_weights = []

for subdir in os.listdir(root_dir):
    full_path = os.path.join(root_dir, subdir, "weights", "best.pt")
    if os.path.isfile(full_path):
        best_weights.append(full_path)

# Print all found paths
for path in best_weights:
    print(path)


# Paths
image_dir = "dataset/UAVDT-processed/val/images"
output_img_dir = "dataset/UAVDT-new/images"
output_lbl_dir = "dataset/UAVDT-new/labels"

# Create directories if they don't exist
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# Load model
model = YOLO(path)

# Process each image
for img_file in tqdm(sorted(os.listdir(image_dir))):
    if not img_file.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, img_file)
    results = model(img_path, conf=0.25)

    # Get image dimensions
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Save prediction as YOLO .txt
    label_path = os.path.join(output_lbl_dir, img_file.replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        for box in results[0].boxes.data.tolist():
            cls, x1, y1, x2, y2, conf = int(box[5]), *box[:4], box[4]
            xc = (x1 + x2) / 2 / w
            yc = (y1 + y2) / 2 / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            f.write(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

    # Copy image to new images folder
    shutil.copy(img_path, os.path.join(output_img_dir, img_file))

print("✅ New dataset created at: dataset/UAVDT-new/val/")


runs/train\yolov8-uavdt3\weights\best.pt


  0%|          | 0/1101 [00:00<?, ?it/s]


image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img000001.jpg: 352x640 (no detections), 60.0ms
Speed: 1.9ms preprocess, 60.0ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)


  0%|          | 1/1101 [00:01<23:21,  1.27s/it]


image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img000002.jpg: 352x640 (no detections), 37.7ms
Speed: 1.5ms preprocess, 37.7ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img000003.jpg: 352x640 (no detections), 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)


...


image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img001098.jpg: 352x640 (no detections), 50.8ms
Speed: 2.7ms preprocess, 50.8ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img001099.jpg: 352x640 (no detections), 44.9ms
Speed: 2.2ms preprocess, 44.9ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)


100%|█████████▉| 1099/1101 [01:20<00:00, 12.47it/s]


image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img001100.jpg: 352x640 (no detections), 49.0ms
Speed: 2.3ms preprocess, 49.0ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 c:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_Yolo\dataset\UAVDT-processed\val\images\img001101.jpg: 352x640 (no detections), 45.8ms
Speed: 2.2ms preprocess, 45.8ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)


100%|██████████| 1101/1101 [01:20<00:00, 13.68it/s]

✅ New dataset created at: dataset/UAVDT-new/val/


In [3]:
# !pip install mean_average_precision

In [15]:
import os
import numpy as np
from tqdm import tqdm
from mean_average_precision import MetricBuilder

# Directories
gt_dir = "dataset/UAVDT-processed/val/labels"
pred_dir = "dataset/UAVDT-new/labels"

# Classes
class_names = ['car', 'truck', 'bus', 'vehicle']
num_classes = len(class_names)

# Dummy image size (YOLO normalized)
dummy_w, dummy_h = 640, 480

# Convert YOLO (xc, yc, w, h) to (xmin, ymin, xmax, ymax)
def to_xyxy(box, w_img=1, h_img=1):
    xc, yc, bw, bh = box[:4]
    xmin = (xc - bw / 2) * w_img
    xmax = (xc + bw / 2) * w_img
    ymin = (yc - bh / 2) * h_img
    ymax = (yc + bh / 2) * h_img
    return [xmin, ymin, xmax, ymax]

# Initialize metric
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=num_classes)

# Iterate files
for label_file in tqdm(os.listdir(gt_dir)):
    if not label_file.endswith(".txt"):
        continue

    gt_path = os.path.join(gt_dir, label_file)
    pred_path = os.path.join(pred_dir, label_file)

    gt_boxes = []
    pred_boxes = []

    # Load ground truth
    with open(gt_path, "r") as f:
        for line in f:
            cls, xc, yc, bw, bh = map(float, line.strip().split())
            box = to_xyxy([xc, yc, bw, bh], dummy_w, dummy_h)
            gt_boxes.append([*box, int(cls), 0, 0])  # Add difficult=0, crowd=0

    # Load predictions
    if os.path.exists(pred_path):
        with open(pred_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                cls = int(parts[0])
                xc, yc, bw, bh = map(float, parts[1:5])
                conf = float(parts[5]) if len(parts) == 6 else 1.0
                box = to_xyxy([xc, yc, bw, bh], dummy_w, dummy_h)
                pred_boxes.append([*box, conf, cls])

    # Add to metric
    metric_fn.add(np.array(pred_boxes), np.array(gt_boxes))

# Compute evaluation results
results = metric_fn.value(iou_thresholds=[0.5], recall_thresholds=np.linspace(0.0, 1.0, 11))

print("\n📊 Evaluation Results (YOLOv8 Predictions vs GT):")
for cls_id, class_name in enumerate(class_names):
    try:
        ap50 = results[0.5][cls_id]["ap"]
        print(f"{class_name:<10}: AP@0.5 = {ap50:.4f}")
    except:
        print(f"{class_name:<10}: AP@0.5 = N/A")

print(f"\n🔹 Mean mAP@0.5: {results['mAP']:.4f}")



100%|██████████| 1101/1101 [00:00<00:00, 2126.47it/s]


📊 Evaluation Results (YOLOv8 Predictions vs GT):
car       : AP@0.5 = 0.0000
truck     : AP@0.5 = 0.0000
bus       : AP@0.5 = 0.0000
vehicle   : AP@0.5 = 0.0000

🔹 Mean mAP@0.5: 0.0000


# Save back into same dataset UAVDT-processed

In [ ]:
# import os
# import cv2
# from ultralytics import YOLO
# from tqdm import tqdm
# import shutil
# import os

# root_dir = "runs/train"

# best_weights = []

# for subdir in os.listdir(root_dir):
#     full_path = os.path.join(root_dir, subdir, "weights", "best.pt")
#     if os.path.isfile(full_path):
#         best_weights.append(full_path)

# # Print all found paths
# for path in best_weights:
#     print(path)


# # Paths
# image_dir = "dataset/UAVDT-processed/val/images"
# output_img_dir = "dataset/UAVDT-processed/train/images"
# output_lbl_dir = "dataset/UAVDT-processed/train/labels"

# # Create directories if they don't exist
# os.makedirs(output_img_dir, exist_ok=True)
# os.makedirs(output_lbl_dir, exist_ok=True)

# # Load model
# model = YOLO(path)

# # Process each image
# for img_file in tqdm(sorted(os.listdir(image_dir))):
#     if not img_file.endswith(".jpg"):
#         continue

#     img_path = os.path.join(image_dir, img_file)
#     results = model(img_path, conf=0.25)

#     # Get image dimensions
#     img = cv2.imread(img_path)
#     h, w = img.shape[:2]

#     # Save prediction as YOLO .txt
#     label_path = os.path.join(output_lbl_dir, img_file.replace(".jpg", ".txt"))
#     with open(label_path, "w") as f:
#         for box in results[0].boxes.data.tolist():
#             cls, x1, y1, x2, y2, conf = int(box[5]), *box[:4], box[4]
#             xc = (x1 + x2) / 2 / w
#             yc = (y1 + y2) / 2 / h
#             bw = (x2 - x1) / w
#             bh = (y2 - y1) / h
#             f.write(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

#     # Copy image to new images folder
#     shutil.copy(img_path, os.path.join(output_img_dir, img_file))

# print("✅ New dataset created at: dataset/UAVDT-new/val/")


# Predcition on videos 

In [18]:
import cv2
import os
import torch
import numpy as np
from pathlib import Path

# ------------------- CONFIG -------------------
VIDEO_PATH = "video.mp4"
OUTPUT_DIR = "video_output"
CONF_THRESHOLD = 0.3
FRAME_INTERVAL = 10
# ----------------------------------------------

# Check if video file exists and can be opened
if not os.path.exists(VIDEO_PATH):
    raise FileNotFoundError(f"[ERROR] Video file not found: {VIDEO_PATH}")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"[ERROR] Cannot open video file: {VIDEO_PATH}")
cap.release()


# Assumes 'model' is already loaded and ready
model.conf = CONF_THRESHOLD  # Optional: set confidence threshold

frame_dir = os.path.join(OUTPUT_DIR, "frames")
pred_dir = os.path.join(OUTPUT_DIR, "predictions")
draw_dir = os.path.join(OUTPUT_DIR, "annotated_frames")
os.makedirs(frame_dir, exist_ok=True)
os.makedirs(pred_dir, exist_ok=True)
os.makedirs(draw_dir, exist_ok=True)

# Step 1: Extract frames
def extract_frames(video_path, output_folder, every_n=1):
    cap = cv2.VideoCapture(video_path)
    frame_idx, save_idx = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        if frame_idx % every_n == 0:
            out_path = os.path.join(output_folder, f"frame_{save_idx:05}.jpg")
            cv2.imwrite(out_path, frame)
            save_idx += 1
        frame_idx += 1
    cap.release()

# Step 2: Run model on each frame and save predictions
def infer_and_save(model, frames_folder, pred_folder):
    image_paths = sorted(Path(frames_folder).glob("*.jpg"))
    for path in image_paths:
        img = cv2.imread(str(path))
        h, w = img.shape[:2]
        results = model(img)[0]
        preds = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        txt_path = os.path.join(pred_folder, path.stem + ".txt")
        with open(txt_path, 'w') as f:
            for i, (x1, y1, x2, y2) in enumerate(preds):
                cls = int(classes[i])
                x_center = ((x1 + x2) / 2) / w
                y_center = ((y1 + y2) / 2) / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                f.write(f"{cls} {x_center} {y_center} {bw} {bh}\n")


# Step 3: Draw bounding boxes on frames
def draw_results(frames_folder, pred_folder, draw_folder):
    for path in sorted(Path(frames_folder).glob("*.jpg")):
        img = cv2.imread(str(path))
        h, w = img.shape[:2]
        txt_file = os.path.join(pred_folder, path.stem + ".txt")
        if os.path.exists(txt_file):
            with open(txt_file) as f:
                for line in f:
                    cls, xc, yc, bw, bh = map(float, line.strip().split())
                    x1 = int((xc - bw / 2) * w)
                    y1 = int((yc - bh / 2) * h)
                    x2 = int((xc + bw / 2) * w)
                    y2 = int((yc + bh / 2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(img, f"{class_id_to_name[int(cls)]}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
        cv2.imwrite(os.path.join(draw_folder, path.name), img)

# Step 4: Rebuild video
def rebuild_video(input_folder, output_path, fps=20):
    files = sorted(Path(input_folder).glob("*.jpg"))
    if not files: return
    first = cv2.imread(str(files[0]))
    h, w = first.shape[:2]
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    for file in files:
        frame = cv2.imread(str(file))
        out.write(frame)
    out.release()

# ------------------- RUN -------------------
if __name__ == "__main__":
    print("[INFO] Extracting frames...")
    extract_frames(VIDEO_PATH, frame_dir, every_n=FRAME_INTERVAL)

    print("[INFO] Running inference...")
    infer_and_save(model, frame_dir, pred_dir)

    print("[INFO] Drawing results...")
    draw_results(frame_dir, pred_dir, draw_dir)

    print("[INFO] Creating final video...")
    rebuild_video(draw_dir, os.path.join(OUTPUT_DIR, "labeled_video.mp4"))

    print("[DONE] Video labeling complete!")


[INFO] Extracting frames...
[INFO] Running inference...

0: 384x640 (no detections), 61.0ms
Speed: 1.8ms preprocess, 61.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.8ms
Speed: 2.9ms preprocess, 44.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.8ms
Speed: 2.1ms preprocess, 45.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.1ms
Speed: 2.4ms preprocess, 44.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.1ms
Speed: 2.4ms preprocess, 46.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.2ms
Speed: 2.3ms preprocess, 46.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.9ms
Speed: 2.3ms preprocess, 44.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (

# Save predciton again on same directory 

In [33]:
# ------------------- CONFIG ------------------- 
VIDEO_PATH = "video.mp4" 
FRAME_INTERVAL = 20
CONF_THRESHOLD = 0.3

# Updated paths for retraining
IMAGE_SAVE_DIR = "dataset/UAVDT-new/images"
LABEL_SAVE_DIR = "dataset/UAVDT-new/labels"
# ----------------------------------------------

# Assumes 'model' is already loaded and ready
model.conf = CONF_THRESHOLD  # Optional: set confidence threshold

os.makedirs(IMAGE_SAVE_DIR, exist_ok=True)
os.makedirs(LABEL_SAVE_DIR, exist_ok=True)


In [37]:
# Step 1: Extract frames
def extract_frames(video_path, output_folder, every_n=1):
    cap = cv2.VideoCapture(video_path)
    frame_idx, save_idx = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        if frame_idx % every_n == 0:
            out_path = os.path.join(output_folder, f"frame_{save_idx:05}.jpg")
            cv2.imwrite(out_path, frame)
            save_idx += 1
        frame_idx += 1
    cap.release()

def infer_and_save(model, frames_folder, label_folder):
    image_paths = sorted(Path(frames_folder).glob("*.jpg"))
    for path in image_paths:
        img = cv2.imread(str(path))
        h, w = img.shape[:2]
        results = model(img)[0]
        preds = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        txt_path = os.path.join(label_folder, path.stem + ".txt")
        
        with open(txt_path, 'w') as f:
            for i, (x1, y1, x2, y2) in enumerate(preds):
                cls = int(classes[i])
                x_center = ((x1 + x2) / 2) / w
                y_center = ((y1 + y2) / 2) / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                f.write(f"{cls} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

        # Optional: Copy image only if not already in destination
        dst_path = os.path.join(IMAGE_SAVE_DIR, path.name)
        if str(path.resolve()) != str(Path(dst_path).resolve()):
            shutil.copy(str(path), dst_path)


In [38]:
print("[INFO] Extracting frames...")
extract_frames(VIDEO_PATH, IMAGE_SAVE_DIR, every_n=FRAME_INTERVAL)

print("[INFO] Running inference...")
infer_and_save(model, IMAGE_SAVE_DIR, LABEL_SAVE_DIR)

print("[DONE] Saved images to:", IMAGE_SAVE_DIR)
print("[DONE] Saved YOLO-format labels to:", LABEL_SAVE_DIR)


[INFO] Extracting frames...
[INFO] Running inference...

0: 384x640 4 cars, 50.5ms
Speed: 1.7ms preprocess, 50.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 46.5ms
Speed: 2.4ms preprocess, 46.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 44.2ms
Speed: 2.4ms preprocess, 44.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 45.9ms
Speed: 2.0ms preprocess, 45.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 45.5ms
Speed: 2.2ms preprocess, 45.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 43.3ms
Speed: 2.0ms preprocess, 43.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 48.7ms
Speed: 2.1ms preprocess, 48.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 49.2ms
Speed: 2.1ms preprocess, 49.2ms inference, 0.9ms 

# Retrain the model

In [ ]:
# import os
# from ultralytics import YOLO

# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# model = YOLO("yolov8n.pt")

# model.train(
#     data="uavdt.yaml",
#     epochs=1,
#     imgsz=640,
#     batch=32,
#     show=False,
#     name="yolov8-uavdt",
#     project="runs/train"
# )


# Print the metrics again 